In [1]:
import numpy as np
import pickle
import scipy
from scipy.optimize import minimize
import math
import os

## SMPL model
root_folder = '/home/yan/Data2/NOMO_Project_Summary/MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/'
import sys
sys.path.append(root_folder+'SMPL/SMPL_python_v.1.0.0/smpl/')
from smpl_webuser.serialization import load_model

smpl_female_model = root_folder + 'SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl'
registred_folder =  '/home/yan/Data2/NOMO_Project_Summary/Data/TC2_scan/registered_mesh/re_name/dataset01/female/'
out_folder = '/home/yan/Data2/NOMO_Project_Summary/MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/optimized_mesh/dataset01/'

# pre-process initial params
param_folder = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/parameter_female/'

In [2]:
def loadTargetV(scanFile):
    with open(scanFile, 'r') as fp:
        content = fp.readlines()
    content = [x.strip() for x in content]

    targetV = []
    for line in content:
        CC = line.split()
        if len(CC) > 0:
            if CC[0] == 'v':
                targetV.append([float(CC[1]), float(CC[2]), float(CC[3])])

    targetV = np.array(targetV)
    targetV = targetV - np.mean(targetV, 0)
    
    return targetV

def loadInitParam(paramFile):
    
    if paramFile == None:
        init_betas = np.zeros(10)
        init_pose = np.zeros(72)
        deg2rad = 1.0 / 180 * math.pi
        init_pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
        init_pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])
    else:
        with open(paramFile, 'rb') as f:
            param_data = pickle.load(f)
        init_pose = param_data[:72]
        init_betas = param_data[72:]
    
    return init_pose, init_betas

def meshDist(param):
    # model.betas[:] = param
    model.betas[:] = param[27:]
    model.pose[48:60] = param[:12]
    model.pose[3:18] = param[12:27]
    
    modelV = model.r
    modelV = modelV - np.mean(modelV, 0)
    # no need to search the corresponding points by KNN or other methods
    dist = np.mean(np.sqrt(np.sum(np.square(modelV - targetV), axis=1))) * 1000
    return dist

In [3]:
# load SMPL model
model = load_model(smpl_female_model)

deg2rad = 1.0 / 180 * math.pi
# bnds = ((-3, 3), (-3, 3), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5))
bnds = ((-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (-90*deg2rad, -30*deg2rad), 
        (-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (30*deg2rad,   90*deg2rad),
        (5*deg2rad,   30*deg2rad), (-30*deg2rad,-5*deg2rad), (0*deg2rad,    10*deg2rad),  
        (5*deg2rad,   30*deg2rad), (5*deg2rad,  30*deg2rad), (-10*deg2rad,   0*deg2rad),  
        (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (0*deg2rad,    15*deg2rad), 
        (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (-15*deg2rad,   0*deg2rad),
        (0*deg2rad,   10*deg2rad), (0*deg2rad,  10*deg2rad), (-5*deg2rad,    5*deg2rad),
        (-10*deg2rad,  0*deg2rad), (-10*deg2rad, 0*deg2rad), (-10*deg2rad,   0*deg2rad), 
        (-10*deg2rad,  0*deg2rad), (0*deg2rad,  10*deg2rad), (0*deg2rad,    10*deg2rad),
        (-3, 3), (-3, 3), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5))

# registered scan list
scan_list = os.listdir(registred_folder)

In [4]:
for scan in scan_list:
    scan_name = scan[:-4]

    targetV = loadTargetV(registred_folder+scan_name+'.obj') # registered mesh, (6890, 3)
    
    paramFile = None    # param_folder + scan_name + '.pkl'                                        
    init_pose, init_betas = loadInitParam(paramFile)         # female does not have the pre-param
    
    # init_param = init_betas
    init_param        = np.zeros(37)
    init_param[:12]   = init_pose[48:60]
    init_param[12:27] = init_pose[3:18]
    init_param[27:]   = init_betas
    
    res = minimize(meshDist, init_param, method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-6, 'disp': True})

    optimal_param = res.x
    print scan_name, " : ", optimal_param[27:]
    
    # save
    # model.betas[:] = optimal_param
    # model.pose[:]  = init_pose
    model.betas[:]    = optimal_param[27:]
    model.pose[48:60] = optimal_param[:12]
    model.pose[3:18]  = optimal_param[12:27]

    with open(out_folder + 'female_02/' + scan_name + '.obj', 'w') as fp:
        for v in model.r: # [m]
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )

    optimal_param = np.zeros(82)
    # optimal_param[:72] = init_pose
    # optimal_param[72:] = optimal_betas
    optimal_param[72:]   = res.x[27:]
    optimal_param[48:60] = res.x[:12]
    optimal_param[3:18]  = res.x[12:27]

    with open(out_folder + 'female_param_02/' + scan_name + '.pkl', 'w') as fp:
        pickle.dump({'param_data': optimal_param}, fp)
        

female_0135  :  [ 0.20297665  0.52551793  1.21182601 -0.02927879 -2.72943127  0.41977232
  0.68389135  3.16678367 -2.72863197  2.6867424 ]
female_0012  :  [-0.88739161 -0.52987551 -0.32923687 -0.23330494  1.41354848 -0.20999197
  2.36988396  0.06246561  3.53748002  1.87461088]
female_0168  :  [ 0.44585881  0.38039854  0.87186088  0.11827965 -0.39719376 -0.32978124
 -0.25289006 -0.07323255 -2.20569257  3.05653989]
female_0164  :  [-1.02516225  0.40636984  0.25831454  0.91002275  1.70623403 -1.96141516
 -0.50708401 -0.15748735 -2.81557816  2.63493836]
female_0094  :  [-0.35758361 -0.54170924 -0.94676715 -0.06733804 -0.78965151  0.94052367
  1.22665322 -0.88974673 -0.52390272  4.99659286]
female_0087  :  [-1.28586878  1.48377656  1.77218039  1.95856755 -2.1439855   1.03066437
  3.42459535  0.64073763 -0.42453834  1.36884185]
female_0036  :  [ 0.39489266  0.66978227  1.57810161 -0.58059738 -4.06644307  2.29041636
  3.65734656  3.39534957  2.22172391  3.60524279]
female_0014  :  [-0.397243 

female_0025  :  [-0.10125567  1.11398405  3.27531061  1.50607279 -0.36383686  0.58784679
  3.44314669 -1.03250238  1.94021777  0.97081537]
female_0045  :  [ 0.01280238  0.48005601 -0.37861601  1.74691305  2.79446954 -3.68474603
 -1.5021569  -1.24181282  4.92980814 -1.65893408]
female_0035  :  [ 0.69469482  0.0397583   2.18377403 -1.76497577  1.04726001  2.59604436
  1.91129223  2.87233438  1.99267986  0.50926547]
female_0157  :  [-1.04941227  1.25781511  1.5892283   0.55714935  0.05742962  0.52700402
  0.24886763  2.01000937 -2.62714821  5.        ]
female_0143  :  [-1.84547163  0.90845068  3.6609877   1.90605184 -1.05645094  0.51898008
  2.23730448  4.19475511 -0.87953819  3.57317785]
female_0119  :  [ 0.69038531  0.44229242  0.7319576   0.89671183 -2.56684422  1.49926076
  2.59290824  1.29240674 -2.6021549   2.28924444]
female_0106  :  [ 1.61750235  1.00743325  1.88581817  0.53757375  1.48458484 -3.95919993
 -1.48416685 -1.92448484  4.5813979  -1.95979835]
female_0024  :  [ 0.0351201

female_0044  :  [-0.25027287  1.12448257  0.21939601 -1.56348338 -1.45464761  4.25809686
  2.53484171  2.07903029  2.15831061  0.36417525]
female_0154  :  [-1.32980007 -0.56752302  0.33481166 -1.52326253  0.35989875  0.00514987
 -0.4453643   3.49989505 -1.7004864   2.82591713]
female_0131  :  [ 0.82468019  0.20838932  3.48268035 -0.06750815 -2.61403271 -0.03809835
  0.10578494  2.75372441 -0.85422888  3.77752308]
female_0170  :  [-1.58782062  1.08492818  1.99431691  1.83793944  2.67061077 -0.21340263
  2.06819126  1.99520324 -0.85011789  3.13215628]
female_0123  :  [-0.49333299  0.62905271  0.86051673 -0.08194315 -2.93542677 -0.21094036
  1.59868822  2.41794505  1.36261894  1.62533808]
female_0029  :  [ 0.387171    1.63710179  2.50883382  0.1488564  -1.24960818  1.01135285
  2.77085688  2.29897506 -0.06656781  2.26279697]
female_0019  :  [-1.44549785 -0.72509173  0.75932911  1.41675858  3.52361754 -0.53160891
  0.76825739 -0.18097004  3.04660894  1.34276076]
female_0105  :  [ 0.3668911

In [5]:
print 'done'

done
